In [1]:
sc.applicationId

u'application_1560417165231_25215'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from pyspark import SparkContext, SparkConf, SparkStageInfo, SparkJobInfo
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import re
from matplotlib.backends.backend_pdf import PdfPages
import csv

#from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression
from pyspark.ml.feature import QuantileDiscretizer, VectorAssembler, Imputer
#from pyspark.ml.evaluation import BinaryClassificationEvaluator

#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import RandomUnderSampler

from sklearn.decomposition import pca
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
# always split into test and train before trying oversampling techniques.
from sklearn.utils import resample
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, recall_score, average_precision_score

In [21]:
import myfunctions as mf
reload(mf)
from myfunctions import *

# 1. First version of features

In [4]:
df_base_full=spark.table('ddp_consumerlending_vault.p901cyo_combined_v1')

## 1.1 information value for all the features

In [127]:
for segment in range(0,2):
    # filter the dataset based on the segment
    df=df_base_full.filter(F.col('poor_payment_flag')==segment)
    print(df.cache().count())
    df.groupby('default').count().show() 
    # feature list
    drop_feature=['default', 'poor_payment_flag', 'partygenid', 'disbursement_month','gender','check','perf_n']
    feature_grp=[x for x in df.columns if x not in drop_feature]
    # calculate information value
    #https://medium.com/@sundarstyles89/weight-of-evidence-and-information-value-using-python-6f05072e83eb
    #define output variables needed
    var_list = []
    missing = []
    zero = []
    unique=[]
    iv_list = []
    df_pd_cal_var = []
    df_pd_cal_cell = []
    df_pd_cal_default_sum = []
    df_pd_cal_default_count = []
    df_pd_cal_bin_max = []
    df_pd_cal_bin_min = []
    #https://stackoverflow.com/questions/21489111/how-to-assign-a-plot-to-a-variable-and-use-the-variable-as-the-return-value-in-a
    #https://www.programcreek.com/python/example/93797/matplotlib.backends.backend_pdf.PdfPages
    # output graphs and values
    pdfname = 'Charts_allvar'+str(segment)+'.pdf'
    with PdfPages(pdfname) as export_pdf:
        
        for i in range(0,len(feature_grp)): 
            df_pd = df.select('default',feature_grp[i]).toPandas()
            n = len(df_pd['default'])
            default_n = sum(df_pd['default'])
            nondefault_n = len(df_pd['default']) - sum(df_pd['default'])
            var = feature_grp[i]
            #print('variable to test: %s' %var)
            var_list.append(var)
            missing.append((df_pd[var].isna().sum()*0.01)/(n*0.01))
            zero.append((len(df_pd[df_pd[var]==0])*0.01)/(n*0.01))
            unique.append(df_pd[var].nunique())

            cell_n = 10

            while cell_n>=1:
                sel_var = ['Decile_rank','default']
                grp_var = ['Decile_rank']
                grp_name = str('Decile_rank')
                try:
                    cell_n = cell_n
                    summary, df_pd_cal = mf.bin_iv_var(df_pd=df_pd, sel_var = sel_var, grp_var = grp_var, grp_name = grp_name, var = var, 
                                                       default_n = default_n, nondefault_n = nondefault_n, cell = cell_n)
                    for j in range(0, len(df_pd_cal)):
                        df_pd_cal_var.append(list(df_pd_cal['var_name'])[j])
                        df_pd_cal_cell.append(list(df_pd_cal[grp_name])[j])
                        df_pd_cal_default_sum.append(list(df_pd_cal['default']['sum'])[j])
                        df_pd_cal_default_count.append(list(df_pd_cal['default']['count'])[j])
                        df_pd_cal_bin_max.append(list(df_pd_cal[var]['max'])[j])
                        df_pd_cal_bin_min.append(list(df_pd_cal[var]['min'])[j])
                    fig = mf.bin_plot(data=summary, var=var)
                    export_pdf.savefig(figure=fig)
                    iv_list.append(sum(df_pd_cal['IV']))
                    break
                except ValueError:
                    cell_n = cell_n-1
                    continue
                except TypeError:
                    print('%s something is wrong with the graph' %var)
                    iv_list.append('error')
                    break
                except KeyboardInterrupt:
                    print('I want to stop this')
                    break
    ivfeature = "IV_feature_check_allvar_"+str(segment)
    ivfeaturecsv = "IV_feature_check_allvar_"+str(segment)+".csv"

    ivfeature = pd.DataFrame({'var': var_list,
                              'missing': missing,
                              'zero': zero,
                              'unique': unique,
                              'iv': iv_list})
    ivfeature.to_csv(ivfeaturecsv, sep=',', encoding='utf-8')
#https://www.programiz.com/python-programming/working-csv-files

243613
+-------+------+
|default| count|
+-------+------+
|      1|   467|
|      0|243146|
+-------+------+

variable to test: duration_poor_payment_count
variable to test: duration_poor_payment_distinct
variable to test: max_severity_class
variable to test: mean_severity_class
variable to test: n_card_trx_all
variable to test: amt_card_trx_all
variable to test: n_card_trx_f
variable to test: amt_card_trx_f
variable to test: n_currency
variable to test: age
variable to test: TradingCapitalMarkets_avg_amt
variable to test: ISKInvestmentSavingsAccount_avg_amt
variable to test: CreditCards_avg_amt
variable to test: DebitCards_avg_amt
variable to test: ConsumerLending_avg_amt
variable to test: DirectInvestmentsinMutualFunds_avg_amt
variable to test: PrivateResidentialLending_avg_amt
variable to test: PensionSavings_avg_amt
variable to test: SavingsAccounts_avg_amt
variable to test: TransactionAccounts_avg_amt
variable to test: months_of_collector_trx
variable to test: nr_of_collector_trx


## 1.2 correlation matrix

In [120]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, threshold):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    au_corr = au_corr[au_corr > threshold]
    
    au_corr_df = au_corr.to_frame().reset_index()
    au_corr_df = au_corr_df.rename(columns ={'level_0': 'feature_1',
                                             'level_1': 'feature_2',
                                              0: 'correlation'})
    print 'The number of highly correlated feature pairs is: ', au_corr_df['correlation'].count()
    return au_corr_df

In [121]:
import numpy as np
pdf = (df_base_full.fillna(0).filter(F.col('poor_payment_flag')==1)
                    .drop('partygenid','disbursement_month','poor_payment_flag', 'default').toPandas())
# calculate the correlations
corr = get_top_abs_correlations(pdf, 0.8)
corr.to_csv('correlationmatrix.csv', sep=',', encoding='utf-8')

The number of highly correlated feature pairs is:  9045


## 1.3 selected features from 1.1 and 1.2

In [5]:
select_var=['partygenid', 'disbursement_month', 'poor_payment_flag', 'default', 'duration_poor_payment_count', 
            'duration_poor_payment_distinct', 'max_severity_class', 'mean_severity_class', 'n_card_trx_all', 
            'amt_card_trx_all', 'n_card_trx_f', 'amt_card_trx_f', 'n_currency', 'age', 'TradingCapitalMarkets_avg_amt',
            'ISKInvestmentSavingsAccount_avg_amt', 'CreditCards_avg_amt', 'DebitCards_avg_amt', 'ConsumerLending_avg_amt',
            'DirectInvestmentsinMutualFunds_avg_amt', 'PrivateResidentialLending_avg_amt', 'PensionSavings_avg_amt', 
            'SavingsAccounts_avg_amt', 'TransactionAccounts_avg_amt', 'nr_of_collector_trx', 'collector_trx_amount', 
            'gambling_flag_last_12_months', 'NumOfTransNight_last_6_months', 'NumOfTransDay_last_6_months', 
            'AmountOfTransNight_last_6_months', 'AmountOfTransDay_last_6_months', 'income_amt_6_std', 'income_amt_6_avg', 
            'income_n_6_avg', 'nr_totalproducts', 'nr_savingsInsuranceproducts', 'nr_Lendingproducts',
            'Trading&CapitalMarkets', 'ISK-InvestmentSavingsAccount', 'CreditCards', 'DebitCards', 'ConsumerLending', 
            'DirectInvestmentsinMutualFunds', 'PrivateResidentialLending', 'PensionSavings', 'SavingsAccounts', 'Swish', 
            'TransactionAccounts', 'Nyckelkund', 'TelebankPersonligS', 'Internetbetalning', 'Grundavtalinternet', 
            'NumOfPlannedTransfersTotalMonth_avg_last_6m', 'AmountOfTotalMonthlyPlannedTransfers_avg_last_6m',
            'TotalNumOfDepositsMonth_avg_last_6m', 'NumOfATMDepositsMonth_avg_last_6m', 'AmountOfATMDepositsMonth_avg_last_6m', 
            'AmountOfTotalWithdrawalsMonth_avg_last_6m', 'NumOfATMWithdrawalsMonth_avg_last_6m', 
            'AmountOfATMWithdrawalsMonth_avg_last_6m', 'NumOfTransactionsActiveCustomer_avg_last_6m', 
            'ConsumerLoansCompetitors_flag_last_6_months', 'invest_and_saving_n_amt', 'lending_n_amt', 'trx_n_amt',
            'invest_and_saving_sum_amt', 'lending_sum_amt', 'trx_sum_amt']

### 1.3.1 different ways of filling in missing

In [6]:
#df_base = spark.table('ddp_consumerlending_vault.p901cyo_combined_v2')
df_base = df_base_full.select(select_var)
# cache df_base
print(df_base.cache().count())

345852


In [7]:
# one way to fill the missings--> all with 0
df_base_fillna = df_base.fillna(0)

In [8]:
drop_feature=['default', 'poor_payment_flag', 'partygenid', 'disbursement_month']
feature_grp=[x for x in df_base.columns if x not in drop_feature]

feature_numeric = [col for col in feature_grp if (df_base.select(col).dropna().distinct().count()>10)]
print(len(feature_numeric),feature_numeric)

(46, ['duration_poor_payment_count', 'duration_poor_payment_distinct', 'mean_severity_class', 'n_card_trx_all', 'amt_card_trx_all', 'n_card_trx_f', 'amt_card_trx_f', 'n_currency', 'age', 'TradingCapitalMarkets_avg_amt', 'ISKInvestmentSavingsAccount_avg_amt', 'CreditCards_avg_amt', 'DebitCards_avg_amt', 'ConsumerLending_avg_amt', 'DirectInvestmentsinMutualFunds_avg_amt', 'PrivateResidentialLending_avg_amt', 'PensionSavings_avg_amt', 'SavingsAccounts_avg_amt', 'TransactionAccounts_avg_amt', 'nr_of_collector_trx', 'collector_trx_amount', 'NumOfTransNight_last_6_months', 'NumOfTransDay_last_6_months', 'AmountOfTransNight_last_6_months', 'AmountOfTransDay_last_6_months', 'income_amt_6_std', 'income_amt_6_avg', 'income_n_6_avg', 'nr_totalproducts', 'nr_savingsInsuranceproducts', 'nr_Lendingproducts', 'NumOfPlannedTransfersTotalMonth_avg_last_6m', 'AmountOfTotalMonthlyPlannedTransfers_avg_last_6m', 'TotalNumOfDepositsMonth_avg_last_6m', 'NumOfATMDepositsMonth_avg_last_6m', 'AmountOfATMDeposit

In [9]:
# another way to fill the missings -->medium
df_base_cast = df_base.select('default', 'poor_payment_flag', 'partygenid', 'disbursement_month', *(F.col(c).cast("float").alias(c) for c in feature_grp))
#df_base_cast = df_base.select( *(F.col(c).cast("float").alias(c) for c in feature_grp))
feature_numeric_out = [s + '_out' for s in feature_numeric]
imputer = Imputer(inputCols=feature_numeric, outputCols=feature_numeric)
df_base_fillmedian = imputer.setStrategy("median").setMissingValue(0).fit(df_base_cast).transform(df_base_cast)
df_base_fillm = df_base_fillmedian.fillna(0)

### 1.3.2 information value for different ways of filling missing values

In [30]:
for segment in range(0,2):
    for basedata in (df_base, df_base_fillna, df_base_fillm):
        # filter the dataset based on the segment
        df=basedata.filter(F.col('poor_payment_flag')==segment)
        print(df.cache().count())
        df.groupby('default').count().show() 
        # feature list
        drop_feature=['default', 'poor_payment_flag', 'partygenid', 'disbursement_month','gender','check','perf_n']
        feature_grp=[x for x in df.columns if x not in drop_feature]
        # calculate information value
        #https://medium.com/@sundarstyles89/weight-of-evidence-and-information-value-using-python-6f05072e83eb
        #define output variables needed
        var_list = []
        missing = []
        zero = []
        unique=[]
        iv_list = []
        df_pd_cal_var = []
        df_pd_cal_cell = []
        df_pd_cal_default_sum = []
        df_pd_cal_default_count = []
        df_pd_cal_bin_max = []
        df_pd_cal_bin_min = []
        #https://stackoverflow.com/questions/21489111/how-to-assign-a-plot-to-a-variable-and-use-the-variable-as-the-return-value-in-a
        #https://www.programcreek.com/python/example/93797/matplotlib.backends.backend_pdf.PdfPages
        # output graphs and values
        if basedata == df_base:
            dataname = str('nofill')
        elif basedata == df_base_fillna:
            dataname = str('fillna')
        elif basedata == df_base_fillm:
            dataname = str('fillm')
            
        pdfname = 'Charts_'+str(dataname)+str(segment)+'.pdf'
        with PdfPages(pdfname) as export_pdf:

            for i in range(0,len(feature_grp)): 
                df_pd = df.select('default',feature_grp[i]).toPandas()
                n = len(df_pd['default'])
                default_n = sum(df_pd['default'])
                nondefault_n = len(df_pd['default']) - sum(df_pd['default'])
                var = feature_grp[i]
                #print('variable to test: %s' %var)
                var_list.append(var)
                missing.append((df_pd[var].isna().sum()*0.01)/(n*0.01))
                zero.append((len(df_pd[df_pd[var]==0])*0.01)/(n*0.01))
                unique.append(df_pd[var].nunique())

                cell_n = 10

                while cell_n>=1:
                    sel_var = ['Decile_rank','default']
                    grp_var = ['Decile_rank']
                    grp_name = str('Decile_rank')
                    try:
                        cell_n = cell_n
                        summary, df_pd_cal = mf.bin_iv_var(df_pd=df_pd, sel_var = sel_var, grp_var = grp_var, grp_name = grp_name, var = var, 
                                                           default_n = default_n, nondefault_n = nondefault_n, cell = cell_n)
                        for j in range(0, len(df_pd_cal)):
                            df_pd_cal_var.append(list(df_pd_cal['var_name'])[j])
                            df_pd_cal_cell.append(list(df_pd_cal[grp_name])[j])
                            df_pd_cal_default_sum.append(list(df_pd_cal['default']['sum'])[j])
                            df_pd_cal_default_count.append(list(df_pd_cal['default']['count'])[j])
                            df_pd_cal_bin_max.append(list(df_pd_cal[var]['max'])[j])
                            df_pd_cal_bin_min.append(list(df_pd_cal[var]['min'])[j])
                        fig = mf.bin_plot(data=summary, var=var)
                        export_pdf.savefig(figure=fig)
                        iv_list.append(sum(df_pd_cal['IV']))
                        break
                    except ValueError:
                        cell_n = cell_n-1
                        continue
                    except TypeError:
                        print('%s something is wrong with the graph' %var)
                        iv_list.append('error')
                        break
                    except KeyboardInterrupt:
                        print('I want to stop this')
                        break
        ivfeature = "IV_feature_check_"+str(dataname)+str(segment)
        ivfeaturecsv = "IV_feature_check_"+str(dataname)+str(segment)+".csv"

        ivfeature = pd.DataFrame({'var': var_list,
                                  'missing': missing,
                                  'zero': zero,
                                  'unique': unique,
                                  'iv': iv_list})
        ivfeature.to_csv(ivfeaturecsv, sep=',', encoding='utf-8')
        
        binfeature = "feature_bin_"+str(dataname)+str(segment)
        binfeaturecsv = "feature_bin_"+str(dataname)+str(segment)+".csv"
        binfeature = pd.DataFrame({'var': df_pd_cal_var,
                                   'cell': df_pd_cal_cell,
                                   'default': df_pd_cal_default_sum,
                                   'total': df_pd_cal_default_count,
                                   'bin_max': df_pd_cal_bin_max,
                                   'bin_min':df_pd_cal_bin_min})
        binfeature.to_csv(binfeaturecsv, sep=',', encoding='utf-8')
#         with open(binfeature, 'w') as writeFile:
#             writer = csv.writer(writeFile)
#             for i in range(0,len(iv_list)):
#                 writer.writerow([df_pd_cal_var[i], df_pd_cal_cell[i],  df_pd_cal_default_sum[i],  df_pd_cal_default_count[i], df_pd_cal_bin_max[i], df_pd_cal_bin_min[i]])
#         writeFile.close()
        print(str(segment), dataname, len(var_list))

243613
+-------+------+
|default| count|
+-------+------+
|      1|   467|
|      0|243146|
+-------+------+

('0', 'nofill', 64)
243613
+-------+------+
|default| count|
+-------+------+
|      1|   467|
|      0|243146|
+-------+------+

('0', 'fillna', 64)
243613
+-------+------+
|default| count|
+-------+------+
|      1|   467|
|      0|243146|
+-------+------+

('0', 'fillm', 64)
102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'nofill', 64)
102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'fillna', 64)
102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'fillm', 64)


### 1.3.3 Calculate IV for the important variables from different missing filling method with decision tree binning

In [22]:
for segment in range(0,2):
    for basedata in (df_base_fillna, df_base_fillm):
        # filter the dataset based on the segment
        df=basedata.filter(F.col('poor_payment_flag')==segment)
        print(df.cache().count())
        df.groupby('default').count().show() 
        
        if basedata == df_base:
            dataname = str('nofill')
        elif basedata == df_base_fillna:
            dataname = str('fillna')
        elif basedata == df_base_fillm:
            dataname = str('fillm')
        
        ivfeature = "IV_feature_check_"+str(dataname)+str(segment)+".csv"        
        feature = mf.getfeature(file_name = ivfeature)
        
        df_pd_cal_var = []
        df_pd_cal_iv = []
        
        for i in range(0,len(feature)): 
            #print(feature[i])
            result_df = bin_iv_var2(df=df, var=feature[i]) 
            df_pd_cal_var.append(feature[i])
            df_pd_cal_iv.append(result_df['iv'].sum())
                
        dtfeature = "IV_feature_check_"+str(dataname)+str(segment)+"_dt"+".csv"
        with open(dtfeature, 'w') as writeFile:
            writer = csv.writer(writeFile)
            for i in range(0,len(df_pd_cal_var)):
                writer.writerow([df_pd_cal_var[i], df_pd_cal_iv[i]])
        writeFile.close()
        print(str(segment), dataname)

243613
+-------+------+
|default| count|
+-------+------+
|      1|   467|
|      0|243146|
+-------+------+

('0', 'fillna')
243613
+-------+------+
|default| count|
+-------+------+
|      1|   467|
|      0|243146|
+-------+------+

('0', 'fillm')
102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'fillna')
102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'fillm')


In [23]:
segment = 1
basedata = df_base_fillna
df=basedata.filter(F.col('poor_payment_flag')==segment)
print(df.cache().count())
#df.groupby('default').count().show() 
        
if basedata == df_base:
    dataname = str('nofill')
elif basedata == df_base_fillna:
    dataname = str('fillna')
elif basedata == df_base_fillm:
    dataname = str('fillm')

ivfeature = "IV_feature_check_"+str(dataname)+str(segment)+".csv"        
feature = mf.getfeature(file_name = ivfeature)
var = feature[0]

102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+



In [24]:
from sklearn.tree import DecisionTreeClassifier
x = df.select(var).toPandas()  # send to pandas
y = df.select('default').toPandas()
    
clf = DecisionTreeClassifier(criterion='entropy',   
                             max_leaf_nodes=6,       
                             min_samples_leaf=0.05)

clf.fit(x, y)  

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=6, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=0.05,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [26]:
clf.tree_.children_left

array([ 1,  3, -1,  7,  5, -1, -1, -1,  9, -1, -1])

In [27]:
clf.tree_.children_right

array([ 2,  4, -1,  8,  6, -1, -1, -1, 10, -1, -1])

In [28]:
clf.tree_.node_count

11

In [29]:
clf.tree_.threshold

array([ 12.5,   4.5,  -2. ,   2.5,   8.5,  -2. ,  -2. ,  -2. ,   3.5,
        -2. ,  -2. ])

In [ ]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
threshold = clf.tree_.threshold

In [31]:
boundary = optimal_binning_boundary(df=df, var=var) 

In [32]:
boundary

[1, 2.5, 3.5, 4.5, 8.5, 12.5, 210.09999999999999]